In [1]:
import requests
import config
from bs4 import BeautifulSoup as BS
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns



In [2]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
#pip install sodapy

import pandas as pd
from sodapy import Socrata
import config_nyc_od

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cityofnewyork.us", None)


In [ ]:
# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofnewyork.us',
                 config_nyc_od.app_tok,
                 username=config_nyc_od.app_username,
                 password=config_nyc_od.app_pw)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
#results = client.get("tg4x-b46p", limit=2000)

# Convert to pandas DataFrame
#results_df = pd.DataFrame.from_records(results)

In [ ]:
felony_df = pd.read_excel(r'felony_clean.xls')
felony_df['PCT'].fillna( method ='ffill', inplace = True)
felony_df = felony_df[felony_df['CRIME']=='TOTAL SEVEN MAJOR FELONY OFFENSES']
felony_df.head(10)

In [ ]:
other_felony_df = pd.read_excel(r'other_felony.xls')
other_felony_df['PCT'].fillna( method ='ffill', inplace = True)
other_felony_df = other_felony_df[other_felony_df['CRIME']=='TOTAL NON-SEVEN MAJOR FELONY OFFENSES']
other_felony_df.head()

In [ ]:
misdemeanor_df = pd.read_excel(r'misdemeanor_pct.xls')
misdemeanor_df['PCT'].fillna( method = 'ffill', inplace = True)
misdemeanor_df = misdemeanor_df[misdemeanor_df['CRIME']=='TOTAL MISDEMEANOR OFFENSES']
misdemeanor_df.head(20)

In [ ]:
violation_df = pd.read_excel(r'violation_pct.xls')
violation_df['PCT'].fillna( method = 'ffill', inplace = True)
violation_df = violation_df[violation_df['CRIME']=='TOTAL VIOLATION OFFENSES']
violation_df.head(20)

In [ ]:
# felony_df = felony_df.set_index('PCT')
# other_felony_df = other_felony_df.set_index('PCT')
# misdemeanor_df = misdemeanor_df.set_index('PCT')
# violation_df = violation_df.set_index('PCT')

In [ ]:
felony_df

In [ ]:
x = felony_df['PCT']
y = felony_df.iloc[3]

fig, ax = plt.subplots(figsize=(20, 10))
sns.scatterplot(x, y, s=200)
#ax.legend(prop={'size': 15})
plt.title('title', fontsize=25)
plt.xticks(size=15)
plt.xlabel('xlabel', size=25)
plt.ylabel('ylabel', size=25)
plt.show()

In [ ]:
#fig, ax = plt.subplots(figsize=(20, 10))
felony_df2 = felony_df.set_index('PCT')
felony_df2.plot.bar(figsize=(16,8))
#plt.show()

In [4]:
rent_df = pd.read_csv("median_rent.csv")
rent_df.head()

,areaName,Borough,areaType,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,...,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10
0,All Downtown,Manhattan,submarket,3200.0,3200.0,3050.0,3100.0,3100.0,3200.0,3195.0,...,3650.0,3750.0,3735.0,3750.0,3827.0,3800.0,3795.0,3800.0,3850.0,3850.0
1,All Midtown,Manhattan,submarket,2895.0,2800.0,2800.0,2850.0,2900.0,2950.0,3000.0,...,3395.0,3400.0,3392.0,3450.0,3500.0,3525.0,3550.0,3550.0,3573.0,3595.0
2,All Upper East Side,Manhattan,submarket,2469.0,2450.0,2400.0,2500.0,2550.0,2550.0,2595.0,...,2929.0,2900.0,2869.0,2850.0,2850.0,2900.0,2950.0,2900.0,2900.0,3000.0
3,All Upper Manhattan,Manhattan,submarket,1825.0,1800.0,1795.0,1800.0,1823.0,1850.0,1875.0,...,2261.0,2291.0,2300.0,2350.0,2375.0,2395.0,2425.0,2400.0,2350.0,2350.0
4,All Upper West Side,Manhattan,submarket,2895.0,2800.0,2750.0,2800.0,2798.0,2795.0,2800.0,...,3350.0,3290.0,3250.0,3290.0,3300.0,3350.0,3350.0,3348.0,3400.0,3400.0


In [ ]:
rent_df.groupby(level=1).mean()